In [8]:
!pip install pyomo pandas numpy openpyxl -q
!apt-get install -y coinor-cbc -q

import pandas as pd
import numpy as np
from pyomo.environ import *

np.random.seed(0)

products = [f'P{i+1}' for i in range(10)]
levels = [f'L{j+1}' for j in range(4)]

revenue = pd.Series(np.random.randint(10, 50, len(products)), index=products)
space_req = pd.Series(np.random.randint(1, 3, len(products)), index=products)  # smaller space
weight_req = pd.Series(np.random.randint(1, 2, len(products)), index=products)  # light
height_req = pd.Series(np.random.randint(5, 10, len(products)), index=products) # short height
fragility = pd.Series(np.random.randint(0, 2, len(products)), index=products)
min_facings_per_level = pd.Series([0]*len(products), index=products)  # allow zero per level
max_facings_per_level = pd.Series(np.random.randint(3, 6, len(products)), index=products)
min_facings_total = pd.Series([1]*len(products), index=products)
max_facings_total = pd.Series(np.random.randint(4, 8, len(products)), index=products)

level_capacity = pd.Series([50]*len(levels), index=levels)  # big enough
level_weight_limit = pd.Series([100]*len(levels), index=levels)
level_height_limit = pd.Series([50]*len(levels), index=levels)
max_fragile_per_level = 3

model = ConcreteModel()
model.P = Set(initialize=products)
model.L = Set(initialize=levels)
model.x = Var(model.P, model.L, domain=NonNegativeIntegers)
model.y = Var(model.P, model.L, domain=Binary)
model.z = Var(model.P, model.L, domain=NonNegativeReals)

def obj_rule(m):
    return sum(revenue[p] * m.z[p, l] for p in m.P for l in m.L)
model.obj = Objective(rule=obj_rule, sense=maximize)

def space_rule(m, l):
    return sum(space_req[p] * m.x[p, l] for p in m.P) <= level_capacity[l]
model.space_con = Constraint(model.L, rule=space_rule)

def fragility_rule(m, l):
    return sum(fragility[p] * m.y[p, l] for p in m.P) <= max_fragile_per_level
model.fragility_con = Constraint(model.L, rule=fragility_rule)

def min_facing_level_rule(m, p, l):
    return m.x[p, l] >= min_facings_per_level[p] * m.y[p, l]
model.min_facing_level_con = Constraint(model.P, model.L, rule=min_facing_level_rule)

def max_facing_level_rule(m, p, l):
    return m.x[p, l] <= max_facings_per_level[p] * m.y[p, l]
model.max_facing_level_con = Constraint(model.P, model.L, rule=max_facing_level_rule)

def min_facings_total_rule(m, p):
    return sum(m.x[p, l] for l in m.L) >= min_facings_total[p]
model.min_facings_total_con = Constraint(model.P, rule=min_facings_total_rule)

def max_facings_total_rule(m, p):
    return sum(m.x[p, l] for l in m.L) <= max_facings_total[p]
model.max_facings_total_con = Constraint(model.P, rule=max_facings_total_rule)

def weight_rule(m, l):
    return sum(weight_req[p] * m.x[p, l] for p in m.P) <= level_weight_limit[l]
model.weight_con = Constraint(model.L, rule=weight_rule)

def height_rule(m, l):
    return sum(height_req[p] * m.y[p, l] for p in m.P) <= level_height_limit[l]
model.height_con = Constraint(model.L, rule=height_rule)

def z_upper1(m, p, l):
    return m.z[p, l] <= m.x[p, l]
model.z_up1 = Constraint(model.P, model.L, rule=z_upper1)

def z_upper2(m, p, l):
    return m.z[p, l] <= max_facings_per_level[p] * m.y[p, l]
model.z_up2 = Constraint(model.P, model.L, rule=z_upper2)

def z_lower(m, p, l):
    return m.z[p, l] >= m.x[p, l] - max_facings_per_level[p] * (1 - m.y[p, l])
model.z_low = Constraint(model.P, model.L, rule=z_lower)

def assign_rule(m, p, l):
    return m.x[p, l] <= max_facings_per_level[p] * m.y[p, l]
model.assign_con = Constraint(model.P, model.L, rule=assign_rule)

def diversity_rule(m, l):
    return sum(m.y[p, l] for p in m.P) >= 2
model.diversity_con = Constraint(model.L, rule=diversity_rule)

solver = SolverFactory('cbc')
solver.solve(model)

alloc = []
for p in model.P:
    for l in model.L:
        if value(model.x[p, l]) > 0:
            alloc.append([p, l, int(value(model.x[p, l])), int(value(model.y[p, l])), round(value(model.z[p, l]), 2)])

df_alloc = pd.DataFrame(alloc, columns=['Product', 'Level', 'Facings', 'Assigned', 'Z Value'])
print(df_alloc)


Reading package lists...
Building dependency tree...
Reading state information...
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   Product Level  Facings  Assigned  Z Value
0       P1    L4        4         1      4.0
1       P2    L3        4         1      4.0
2       P3    L4        4         1      4.0
3       P4    L2        3         1      3.0
4       P4    L3        2         1      2.0
5       P5    L1        5         1      5.0
6       P6    L2        3         1      3.0
7       P6    L4        3         1      3.0
8       P7    L1        4         1      4.0
9       P8    L4        4         1      4.0
10      P9    L2        3         1      3.0
11      P9    L3        2         1      2.0
12     P10    L1        5         1      5.0
13     P10    L3        2         1      2.0
